### Wavelete coherence 

In [1]:
import os
import sys;sys.path.insert(1,'..')
import numpy  as np 
import xarray as xr

import matplotlib.pyplot  as plt
import matplotlib.patches as patches
import matplotlib
import plot

from util.surrogates import trial_swap_surrogates

from conn.conn_spec  import conn_spec

In [2]:
SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE=plot.set_plot_config()

#### Bias of the wavelet coherence 

Following Lachaux et. al. (2002) the bias fo the wavelet coherence can be computed as:

$B = \left(1 + 2\frac{n_{cy}}{n_{co}}\right)^{-1}$

where, $n_{cy}$ is the number of cycles of the wavelet within the integration window $\delta$, and $n_{co}$ the number of cycles of the wavelet. The integration window is given by:

$\delta = n_{cy}/f$

Those function could be used to fit the bias of the wavelet coherence computed for white noise signals.

With our terminology, since we input $\delta$ (sm_times) to the function the bias can be written as:

$B = \left(1 + 2\frac{f\delta}{n_{co}}\right)^{-1}$

From this equation it is expected that the bias will be frequency dependent if $n_{co}$ is constant, and frequency independent if $n_{co}$ is proportional to frequency let's say $n_{co}=f/\sigma_f$, then:

$B = \left(1 + 2\delta\sigma_{t}\right)^{-1}$

By comparing the wavelet coherence measured for white noise we can validate the expression above.

In [3]:
def _bias_lachaux(sm_times, freqs, n_cycles):
    return (1+2*sm_times*freqs/n_cycles)**-1

### Bias of the metric - White noise vs. Analytical

In [4]:
n  = 3650
sf = 1000

eta = xr.DataArray( np.random.normal(0,1,size=(1,30,n)), 
                    dims=("trials","roi","times"), 
                    coords={"times": np.arange(n)/sf})

#### Constant $\delta$, and $n_{co}$ constant

In [5]:
delta    = 1
freqs    = np.linspace(10,100,50)
w_t      = np.linspace(0.1, 2., 20)

n_cycles = 7.

kw = dict(
    freqs=freqs, times="times", roi="roi", foi=None, n_jobs=20, pairs=None, sfreq=1000,
    mode="morlet", n_cycles=n_cycles, decim=delta, metric="coh", sm_kernel="square",
    sm_freqs=1, block_size=1
)

coh_eta = []
for i in range( len(w_t) ):
    kw["sm_times"] = w_t[i]
    coh_eta += [conn_spec(eta, **kw)]

Computing pairwise Coherence (n_pairs=435, n_freqs=50, decim=1, sm_times=100, sm_freqs=1)
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  18 out of  30 | elapsed:    4.6s remaining:    3.0s
[Parallel(n_jobs=20)]: Done  25 out of  30 | elapsed:    6.8s remaining:    1.4s
[Parallel(n_jobs=20)]: Done  30 out of  30 | elapsed:    6.9s finished


  0%|          | Estimating pairwise Coherence for trials 0...0 : 0/435 [00:00<?,       ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/vinicius/anaconda3/envs/brainhack/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_840402/3504621058.py", line 16, in <module>
    coh_eta += [conn_spec(eta, **kw)]
  File "../conn/conn_spec.py", line 250, in conn_spec
    conn_tr = conn_f(w, kernel, foi_idx, x_s, x_t, kw_para)
  File "../conn/conn_spec.py", line 50, in _coh
    return parallel(p_fun(s, t) for s, t in zip(x_s, x_t))
  File "/home/vinicius/anaconda3/envs/brainhack/lib/python3.8/site-packages/frites/utils/parallel.py", line 63, in parallel_progress
    return parallel(ProgressBar(iterable=op_iter, max_value=total,
  File "/home/vinicius/anaconda3/envs/brainhack/lib/python3.8/site-packages/joblib/parallel.py", line 1056, in __call__
    self.retrieve()
  File "/home/vinicius/anaconda3/envs/brainhack/lib/python3.8/site-packages/joblib/parallel.py", line 935, in r

TypeError: object of type 'NoneType' has no len()

In [ ]:
B   = []
B_t = []

for i in range( len(w_t) ):
    B += [coh_eta[i].isel(times=slice(20,-20)).squeeze().mean(dim=("roi","times"))]
    #B_t += [(1 + 2*n_cy[i]/n_cycles)**-1 ]
    B_t += [_bias_lachaux(w_t[i], freqs, n_cycles) ]

In [ ]:
plt.figure(figsize=(15,10))
for i in range( len(w_t) ):
    plt.subplot(5,4,i+1)
    plt.title(f"w_t = {np.round(w_t[i],2)}")
    plt.plot(freqs, B[i], '-')
    #plt.plot(freqs, [B_t[i]]*len(freqs), ".")
    plt.plot(freqs, B_t[i], ".")
    plt.ylim(0,1.03)
    if i==0: plt.legend(["White-noise", "Analytical"])
    if i>15: plt.xlabel("Frequency [Hz]")
    if i%4==0: plt.ylabel("Bias")
plt.tight_layout()

#### Constant $\delta$, and $n_{co}$ proportional to frequency

In [ ]:
delta    = 1
freqs    = np.linspace(10,100,50)
w_t      = np.linspace(0.1, 2., 20)

n_cycles = freqs/2.

kw = dict(
    freqs=freqs, times="times", roi="roi", foi=None, n_jobs=20, pairs=None, sfreq=1000,
    mode="morlet", n_cycles=n_cycles, decim=delta, metric="coh", sm_kernel="square",
    sm_freqs=1, block_size=1
)

coh_eta = []
for i in range( len(w_t) ):
    kw["sm_times"] = w_t[i]
    coh_eta += [conn_spec(eta, **kw)]

In [ ]:
B   = []
B_t = []

for i in range( len(w_t) ):
    B += [coh_eta[i].isel(times=slice(20,-20)).squeeze().mean(dim=("roi","times"))]
    #B_t += [(1 + 2*n_cy[i]/n_cycles)**-1 ]
    B_t += [_bias_lachaux(w_t[i], freqs, n_cycles) ]

In [ ]:
plt.figure(figsize=(15,10))
for i in range( len(w_t) ):
    plt.subplot(5,4,i+1)
    plt.title(f"w_t = {np.round(w_t[i],2)}")
    plt.plot(freqs, B[i], '-')
    #plt.plot(freqs, [B_t[i]]*len(freqs), ".")
    plt.plot(freqs, B_t[i], ".")
    plt.ylim(0,1.03)
    if i==0: plt.legend(["White-noise", "Analytical"])
    if i>15: plt.xlabel("Frequency [Hz]")
    if i%4==0: plt.ylabel("Bias")
plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(8,6))
ax  = plt.subplot(111)
B_total   = np.mean(B, -1)
B_t_total = np.mean(B_t, -1)
plt.plot(w_t, B_total, "o", label="white-noise")
plt.plot(w_t, B_t_total, "-", label="analytical")
plt.legend(fontsize=BIGGER_SIZE)
plt.vlines(0.3, 0, 9, "gray", lw=3)
plt.vlines(0.6, 0, 9, "gray", ls="--", lw=3)
plt.vlines(1.0, 0, 9, "gray", ls="-.", lw=3)
plt.ylim(0, .9)
plt.xlim(0.05, 2.05)
plt.ylabel("Bias", fontsize=BIGGER_SIZE)
plt.xlabel(r"$w_t$ [s]", fontsize=BIGGER_SIZE)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tight_layout()

#### Variable $\delta$, and $n_{co}$ constant

In [ ]:
delta    = 1
freqs    = np.linspace(10,100,50)
n_cy     = np.linspace(1, 40, 20)

n_cycles = 7.

kw = dict(
    freqs=freqs, times="times", roi="roi", foi=None, n_jobs=20, pairs=None, sfreq=1000,
    mode="morlet", n_cycles=n_cycles, decim=delta, metric="coh", sm_kernel="square",
    sm_freqs=1, block_size=1
)



coh_eta = []
for i in range( len(n_cy) ):
    # compute the coherence
    kw["sm_times"] = n_cy[i]/freqs
    coh_eta += [conn_spec(eta, **kw)]

In [ ]:
B   = []
B_t = []

for i in range( len(n_cy) ):
    B += [coh_eta[i].isel(times=slice(20,-20)).squeeze().mean(dim=("roi","times"))]
    #B_t += [(1 + 2*n_cy[i]/n_cycles)**-1 ]
    B_t += [_bias_lachaux(n_cy[i]/freqs, freqs, n_cycles) ]

In [ ]:
plt.figure(figsize=(15,10))
for i in range( len(n_cy) ):
    plt.subplot(5,4,i+1)
    plt.title(f"n_cy = {np.round(n_cy[i],2)}")
    plt.plot(freqs, B[i], '-')
    #plt.plot(freqs, [B_t[i]]*len(freqs), ".")
    plt.plot(freqs, B_t[i], ".")
    plt.ylim(0,1.03)
    if i==0: plt.legend(["White-noise", "Analytical"])
    if i>15: plt.xlabel("Frequency [Hz]")
    if i%4==0: plt.ylabel("Bias")
plt.tight_layout()

#### Variable $\delta$, and $n_{co}$ frequency dependent

In [ ]:
delta    = 1
freqs    = np.linspace(10,100,50)
n_cy     = np.linspace(1, 40, 20)

n_cycles = freqs/2.

kw = dict(
    freqs=freqs, times="times", roi="roi", foi=None, n_jobs=20, pairs=None, sfreq=1000,
    mode="morlet", n_cycles=n_cycles, decim=delta, metric="coh", sm_kernel="square",
    sm_freqs=1, block_size=1
)



coh_eta = []
for i in range( len(n_cy) ):
    # compute the coherence
    kw["sm_times"] = n_cy[i]/freqs
    coh_eta += [conn_spec(eta, **kw)]

In [ ]:
B   = []
B_t = []

for i in range( len(n_cy) ):
    B += [coh_eta[i].isel(times=slice(20,-20)).squeeze().mean(dim=("roi","times"))]
    #B_t += [(1 + 2*n_cy[i]/n_cycles)**-1 ]
    B_t += [_bias_lachaux(n_cy[i]/freqs, freqs, n_cycles) ]

In [ ]:
plt.figure(figsize=(15,10))
for i in range( len(n_cy) ):
    plt.subplot(5,4,i+1)
    plt.title(f"n_cy = {np.round(n_cy[i],2)}")
    plt.plot(freqs, B[i], '-')
    #plt.plot(freqs, [B_t[i]]*len(freqs), ".")
    plt.plot(freqs, B_t[i], ".")
    plt.ylim(0,1.03)
    if i==0: plt.legend(["White-noise", "Analytical"])
    if i>15: plt.xlabel("Frequency [Hz]")
    if i%4==0: plt.ylabel("Bias")
plt.tight_layout()